In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain-groq langchain_huggingface faiss-gpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.6 MB/s eta 0

In [36]:
!pip install wikipedia
!pip install -U duckduckgo-search
!pip install langchainhub
!pip install arxiv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.5 MB/s eta 0:00:00


In [80]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [81]:
import os
os.environ['GROQ_API_KEY'] = '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'

In [106]:
llm = ChatOpenAI(                                            # model 1
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,
)

# chat = ChatGroq(model = "mixtral-8x7b-32768",                # model 2
#                 temperature = 0.5,
#                 max_tokens = 5000,
#                 )

# Search from PDF

In [83]:
from langchain.tools.retriever import create_retriever_tool
loader = PyPDFLoader("attenstion_is_all_you_need.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever=vectorstore.as_retriever()
retriever_tool=create_retriever_tool(retriever,"PdfRetriever",
                      "Search the information from the given vectorstore")

# Search From Wikipedia

In [89]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki_search=WikipediaQueryRun(api_wrapper=api_wrapper)

# Search from Arxiv

In [90]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
arxiv_search=ArxivQueryRun(api_wrapper=arxiv_wrapper)

# Search from duckduckgo

In [87]:
from langchain_community.tools import DuckDuckGoSearchRun
duckduckgo_search = DuckDuckGoSearchRun()

# Combine all Tools

In [91]:
tools=[retriever_tool,duckduckgo_search,wiki_search,arxiv_search]

# its a first method for questioning from agent

In [108]:
from langchain.agents import initialize_agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True)
agent.run("What is the paper 1605")



> Entering new AgentExecutor chain...
Thought: The question seems to be asking about a specific paper with the number 1605. This could be a paper from arXiv, a scientific repository.

Action: arxiv
Action Input: 1605
Observation: Published: 2023-10-31
Title: CCD UBV and Gaia DR3 Analyses of Open Clusters King 6 and NGC 1605
Authors: S. Gokmen, Z. Eker, T. Yontan, S. Bilir, T. Ak, S. Ak, T. Banks, A. Sarajedini
Summary: A detailed analysis of ground-based CCD UBV photometry and space-based Gaia
Data Release 3 (DR3) data for the open clusters King 6 and NGC 1605 was
performed. Using the pyUPMASK algorithm on Gaia astrometric data to estimate
cluster membership probabilities, we have identified 112 stars in King 6 and
160 stars in NGC 1605 as the statistically most likely members of each cluster.
We calculated reddening and metallicity separately using UBV two-color diagrams
to estimate parameter values via independent methods. The color excess $E(B-V)$
and photometric metallicity [Fe/H

'The paper 1605 is a scientific paper published on arXiv, titled "CCD UBV and Gaia DR3 Analyses of Open Clusters King 6 and NGC 1605" by S. Gokmen, Z. Eker, T. Yontan, S. Bilir, T. Ak, S. Ak, T. Banks, and A. Sarajedini, published on October 31, 2023.'

# another method for question answering

In [101]:
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
agent=create_openai_tools_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [104]:
responce = agent_executor.invoke({"input":"tell me about code with harry"})
print(responce)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Harry Potter`


Page: Harry Potter
Summary: Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate a
Invoking: `transformer` with `{'query': 'Harry Potter'}`


Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation. arXiv preprint
arXiv:1609.08144 , 2016.
[32] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR 